<a href="https://colab.research.google.com/github/Mateo755/UAV_ML_FDI/blob/main/FDI_UAV_Optuna_(Colab).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UAV Propeller Fault Detection System (Parrot Bebop 2)

This project focuses on the development and validation of fault detection and isolation (FDI) methods for the propulsion system of the **Parrot Bebop 2** unmanned aerial vehicle (UAV). The analysis utilizes inertial sensor data (accelerometer and gyroscope) collected during real-world flight experiments.

### Research Problem
The primary objective is to classify the technical state of the propellers based on vibration signals. We analyze various fault scenarios across four rotors (A, B, C, D), distinguishing between nominal states and specific defects such as chipped edges or bent blades.

### Methodology
This notebook compares two signal processing approaches:
1.  **Time Domain Analysis**
2.  **Frequency Domain Analysis**

Experiment tracking and performance visualization are managed via **Weights & Biases (W&B)**.

# 1. Environment Setup & Global Configuration
Installation of necessary libraries (Weights & Biases for experiment tracking) and importing standard data science modules.

In [ ]:
!pip install wandb
!pip install optuna

In [ ]:
import os
from google.colab import files
import optuna
import tensorflow as tf
import numpy as np
import pandas as pd
import glob
import time
import json
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Input, Flatten, Dense, Dropout,
    Conv1D, MaxPooling1D, GlobalAveragePooling1D, BatchNormalization
)
from tensorflow.keras.optimizers import Adam, RMSprop, SGD, Adagrad, Adadelta, Adamax, Nadam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.metrics import CategoricalAccuracy
import traceback

In [ ]:
import wandb
from wandb.integration.keras import WandbMetricsLogger
wandb.login()

# 2. Exploratory Data Analysis
Initial inspection of the accelerometer and gyroscope data from the UAV. We examine the column structure.

## 2.1 Time domain

In [ ]:
!unzip -q /content/Normalized_data.zip

In [ ]:
df_time = pd.read_csv('/content/Normalized_data/Bebop2_16g_1kdps_normalized_0000.csv')
df_time

In [ ]:
df_time.info()

## 2.2 Frequency Domain

In [ ]:
!unzip -q /content/FFT_data.zip

In [ ]:
df_freq = pd.read_csv('/content/FFT_data/128_Hann_20_52/Bebop2_16g_FFT_ACCEL_128_Hann_20_52_0000.csv', header=None)
df_freq

In [ ]:
df_freq.info()

# 3. Fault Scenario Mapping

The Bebop 2 flight data is labeled using a 4-digit code (e.g., `1022`), defining the state of each propeller (A, B, C, D):
* **0**: Nominal (Functional propeller).
* **1**: Fault Type I (e.g., chipped edge).
* **2**: Fault Type II (e.g., bent tip/severe damage).

Below, we define the mapping of these physical scenarios to model class labels. Depending on the diagnostic granularity required, the problem can be framed as a **5-class problem** (aggregated by the number of faults) or a **20-class problem** (precise fault configuration).

In [ ]:
# Select Experiment Mode
CLASS_MODE = "20class"   # Options: "5class" (aggregated) or "20class" (precise diagnosis)

In [ ]:
# Precise mapping for 20 unique scenarios (Parrot Bebop 2)
scenario_to_class_20 = {
    "0000": 0,  # Nominal state
    "1000": 1, "0100": 2, "0010": 3, "0001": 4,     # Single faults (Type 1)
    "2000": 5, "0200": 6, "0020": 7, "0002": 8,     # Single faults (Type 2)
    "1100": 9, "1020": 10, "1002": 11, "0120": 12, "0102": 13, "0022": 14, # Dual faults
    "1120": 15, "1102": 16, "1022": 17, "0122": 18, # Triple faults
    "1122": 19, # All propellers faulty
}

# Simplified mapping (Number of faulty rotors)
scenario_to_class_5 = {
    "0000": 0,
    "1000": 1, "0100": 1, "0010": 1, "0001": 1,
    "2000": 1, "0200": 1, "0020": 1, "0002": 1,
    "1100": 2, "1020": 2, "1002": 2, "0120": 2, "0102": 2, "0022": 2,
    "1120": 3, "1102": 3, "1022": 3, "0122": 3,
    "1122": 4,
}

if CLASS_MODE == "5class":
    scenario_to_class = scenario_to_class_5
elif CLASS_MODE == "20class":
    scenario_to_class = scenario_to_class_20
else:
    raise ValueError("Invalid CLASS_MODE selected.")

NUM_CLASSES = len(set(scenario_to_class.values()))
print(f"Experiment Mode: {CLASS_MODE} | Total Classes: {NUM_CLASSES}")

# 3. Time Domain Analysis, data preparation

In this section, we process normalized time-series signals from the accelerometers and gyroscopes. Since the data represents a continuous flight stream, we apply a **sliding window** technique to segment the signal into fixed-length samples (e.g., 256 measurement points).



## 3.1 Data Segmentation

In [ ]:
# Path to normalized time-domain data
DOMAIN = "time"
DATA_DIR_TIME = r"/content/Normalized_data"
SAMPLE_SIZE = 8               # Window length
N_FEATURES = 24               # Input channels (e.g., 3-axis accel + 3-axis gyro per sensor)
SENSOR_MODE = "both"          # "accel" + "gyro"

In [ ]:
def make_windows_from_df(df: pd.DataFrame, sample_size: int):
    """
    Segments time-series data into non-overlapping windows.
    """
    data = df.values.astype("float32")
    n_total = len(data)
    n_windows = n_total // sample_size
    if n_windows == 0:
        return np.empty((0, sample_size, data.shape[1]), dtype="float32")
    data = data[:n_windows * sample_size]
    windows = data.reshape(n_windows, sample_size, data.shape[1])
    return windows

### Processing Pipeline
The following loop iterates through all normalized CSV files in time domain. For each file, it:
1.  Extracts the scenario code (e.g., `0000`) from the filename.
2.  Checks if the scenario exists in our defined class mapping.
3.  Loads the data and applies the sliding window segmentation.
4.  Accumulates the processed windows (`X`) and corresponding labels (`y`) into a single dataset.

In [ ]:
# Data pass through window function

X_list = []
y_list = []

norm_data_files_pattern = os.path.join(DATA_DIR_TIME, "Bebop2_16g_1kdps_normalized_*.csv")

for path in glob.glob(norm_data_files_pattern):
    fname = os.path.basename(path)
    # ostatni fragment po "_" to kod scenariusza, np. "0000"
    scenario = os.path.splitext(fname)[0].split("_")[-1]

    if scenario not in scenario_to_class:
        print(f"Pomijam {fname} – scenariusz {scenario} nie jest w mapowaniu.")
        continue

    label = scenario_to_class[scenario]

    df = pd.read_csv(path)

    # jeżeli w pliku są inne kolumny niż 24 sensory, tu można wybrać tylko potrzebne:
    # df = df[["A_aX","A_aY",...,"D_gZ"]]

    windows = make_windows_from_df(df, SAMPLE_SIZE)
    if windows.shape[0] == 0:
        print(f"Za mało danych w {fname} na choć jedno okno, pomijam.")
        continue

    X_list.append(windows)
    y_list.append(np.full((windows.shape[0],), label, dtype="int32"))

X = np.concatenate(X_list, axis=0)  # (N, SAMPLE_SIZE, 24)
y = np.concatenate(y_list, axis=0)  # (N,)

print("X shape:", X.shape)
print("y shape:", y.shape, "unikalne etykiety:", np.unique(y))

input_shape = (SAMPLE_SIZE, N_FEATURES)
print("input_shape modelu:", input_shape)

# 4. Frequency Domain Analysis (FFT)

Mechanical faults in rotating components (such as propellers) generate distinct vibration signatures that are often most discernible in the frequency spectrum.

In this experiment, we utilize data pre-processed via **Fast Fourier Transform (FFT)** using a Hann window to mitigate spectral leakage, which is already done in repo. The input features are vectors of spectral coefficients for each sensor axis.

## 4.1 Spectral Data Structure
The FFT files contain metadata within their filenames (window length, window type, frequency range). The following code parses this information and loads the corresponding spectral coefficients.

In [ ]:
# Configuration for FFT Data
DOMAIN = "fft"
FFT_ROOT      = "FFT_data"
FFT_CONFIG    = "128_Hann_20_52"     # Specific window/range configuration
SENSOR_MODE   = "both"               # "accel", "gyro", or "both"
SAMPLING_RATE = 500.0                # Sampling rate for Bebop 2 inertial sensors

# liczba osi na jeden typ czujnika
N_AXES_SINGLE = 12
N_AXES = 12 if SENSOR_MODE in ("accel", "gyro") else 24

fft_dir = os.path.join(FFT_ROOT, FFT_CONFIG)

In [ ]:
def print_fft_info(fft_dir, sampling_rate=500.0):
    """
    Extracts FFT parameters encoded in the data folder name and converts them
    into physical frequency values.

    The function assumes the folder name follows the format:
    'WindowLength_WindowType_StartBin_StopBin' (e.g., '128_Hann_20_52').
    """

    folder_name = os.path.basename(os.path.normpath(fft_dir))
    parts = folder_name.split("_")          # np. ["128","Hann","20","52"]
    measuringWindowLength = int(parts[0])   # 128
    rangeStart = int(parts[-2])             # 20
    rangeStop  = int(parts[-1])             # 52

    freq_res = sampling_rate / measuringWindowLength
    f_start  = (rangeStart - 1) * freq_res
    f_stop   = rangeStop * freq_res

    print(f"Folder FFT: {folder_name}")
    print(f"measuringWindowLength = {measuringWindowLength}")
    print(f"Zakres binów: {rangeStart}–{rangeStop}")
    print(f"Rozdzielczość częstotliwości: {freq_res:.3f} Hz")
    print(f"Zakres częstotliwości: {f_start:.1f} Hz – {f_stop:.1f} Hz")

    return measuringWindowLength, rangeStart, rangeStop, freq_res, f_start, f_stop

In [ ]:
measuringWindowLength, rangeStart, rangeStop, freq_res, f_start, f_stop = print_fft_info(fft_dir, SAMPLING_RATE)

### File Discovery & Categorization
This block scans the directory for all CSV files and organizes them into dictionaries based on the sensor type (**ACCEL** vs. **GYRO**). It parses the filename to extract the specific fault scenario code (e.g., `0000`, `1022`), using it as a key for quick lookup during the data loading phase.

In [ ]:
# Scan directory and map file paths to scenarios based on sensor type (ACCEL/GYRO)

all_files = glob.glob(os.path.join(fft_dir, "*.csv"))

accel_files = {}  # scenario -> ścieżka
gyro_files  = {}

for path in all_files:
    fname = os.path.basename(path)
    parts = fname.split("_")
    # przykład: Bebop2_16g_FFT_ACCEL_128_Hann_20_52_0000.csv
    # indeksy:   0      1   2   3     4    5    6   7   8
    sensor_type = parts[3]              # "ACCEL" albo "GYRO"
    scenario    = os.path.splitext(parts[-1])[0]  # "0000" itd.

    if sensor_type == "ACCEL":
        accel_files[scenario] = path
    elif sensor_type == "GYRO":
        gyro_files[scenario] = path

print("Znaleziono ACCEL dla scenariuszy:", sorted(accel_files.keys()))
print("Znaleziono GYRO  dla scenariuszy:", sorted(gyro_files.keys()))


### Data Loading & Sensor Fusion
In this step, we aggregate the spectral data based on the selected `SENSOR_MODE`.
* **Accel/Gyro:** Loads only the specified sensor data.
* **Both:** Loads both accelerometer and gyroscope files for the same scenario, verifying consistency, and concatenates them along the feature axis to create a unified feature vector (e.g., 24 input channels).

In [ ]:
def load_fft_file(path, n_axes=N_AXES_SINGLE):
    """
    Loads spectral data from a CSV file and reshapes it into a 3D tensor.

    The function reads a flat CSV (assuming no header), calculates the number
    of frequency bins based on the total columns and specified axes, and
    restructures the data.
    """

    # jeśli okaże się, że plik ma nagłówek – zmień na header=0
    df = pd.read_csv(path, header=None)
    data = df.values.astype("float32")   # (n_okien, n_features)
    n_features = data.shape[1]

    if n_features % n_axes != 0:
        raise ValueError(f"{os.path.basename(path)}: {n_features} kolumn "
                         f"nie dzieli się przez {n_axes} osi.")

    n_freq_bins = n_features // n_axes
    data_3d = data.reshape(-1, n_freq_bins, n_axes)  # (n_okien, n_freq_bins, n_axes)
    return data_3d, n_freq_bins

In [ ]:
X_list = []
y_list = []
n_freq_bins_global = None

for scenario, label in scenario_to_class.items():
    cur_X = None

    if SENSOR_MODE == "accel":
        path = accel_files.get(scenario)
        if path is None:
            print(f"[ACCEL] brak pliku dla scenariusza {scenario}, pomijam.")
            continue
        accel_data, n_freq_bins = load_fft_file(path)
        cur_X = accel_data   # (n_okien, n_freq_bins, 12)

    elif SENSOR_MODE == "gyro":
        path = gyro_files.get(scenario)
        if path is None:
            print(f"[GYRO] brak pliku dla scenariusza {scenario}, pomijam.")
            continue
        gyro_data, n_freq_bins = load_fft_file(path)
        cur_X = gyro_data    # (n_okien, n_freq_bins, 12)

    elif SENSOR_MODE == "both":
        path_a = accel_files.get(scenario)
        path_g = gyro_files.get(scenario)
        if path_a is None or path_g is None:
            print(f"[BOTH] brak ACCEL lub GYRO dla {scenario}, pomijam.")
            continue

        accel_data, n_freq_bins_a = load_fft_file(path_a, n_axes=N_AXES_SINGLE)
        gyro_data,  n_freq_bins_g = load_fft_file(path_g, n_axes=N_AXES_SINGLE)

        if accel_data.shape[0] != gyro_data.shape[0] or n_freq_bins_a != n_freq_bins_g:
            raise ValueError(f"Niezgodne rozmiary ACCEL/GYRO dla scenariusza {scenario}")

        # sklejanie po osi „kanałów”: 12 (ACCEL) + 12 (GYRO) = 24
        cur_X = np.concatenate([accel_data, gyro_data], axis=-1)  # (..., n_freq_bins, 24)
        n_freq_bins = n_freq_bins_a

    else:
        raise ValueError("SENSOR_MODE musi być 'accel', 'gyro' albo 'both'")

    # ustaw / sprawdź globalną liczbę binów
    if n_freq_bins_global is None:
        n_freq_bins_global = n_freq_bins
    elif n_freq_bins_global != n_freq_bins:
        raise ValueError("Różne n_freq_bins między plikami, coś jest nie tak.")

    X_list.append(cur_X)
    y_list.append(np.full((cur_X.shape[0],), label, dtype="int32"))

# Sklejenie wszystkiego
X = np.concatenate(X_list, axis=0)   # (N, n_freq_bins, N_AXES)
y = np.concatenate(y_list, axis=0)   # (N,)

print("X shape:", X.shape)
print("y shape:", y.shape, "unikalne etykiety:", np.unique(y))

n_freq_bins = n_freq_bins_global
input_shape = (n_freq_bins, N_AXES)
print("input_shape modelu:", input_shape)


# 5. OPTUNA

## 5.1 MLP Model Builder

In [ ]:
# --- Cell: MLP Model Builder ---
def MLPBuilder(optimizerStr, dropout, applyDropout, learningRate, hidden_layers_structure):
    """
    Builds a Multi-Layer Perceptron (MLP) model dynamically.

    Args:
        optimizerStr (str): Name of the optimizer to use.
        dropout (float): Dropout rate.
        applyDropout (bool): Whether to apply dropout after dense layers.
        learningRate (float): Learning rate for the optimizer.
        hidden_layers_structure (list): List of integers defining the number of units in each dense layer.
    """
    optimizerCls = {
        "Adam": Adam, "RMSprop": RMSprop, "SGD": SGD,
        "Adagrad": Adagrad, "Adadelta": Adadelta, "Adamax": Adamax, "Nadam": Nadam,
    }

    # 1. Input and Flattening
    layers = [
        Input(shape=input_shape),
        Flatten()
    ]

    # 2. Dynamic Hidden Dense Layers
    for units in hidden_layers_structure:
        layers.append(Dense(units, activation="relu"))
        if applyDropout:
            layers.append(Dropout(dropout))

    # 3. Output Layer
    layers.append(Dense(NUM_CLASSES, activation="softmax"))

    # 4. Model Assembly
    model = Sequential(layers)

    model.compile(
        optimizer=optimizerCls[optimizerStr](learning_rate=learningRate),
        loss="categorical_crossentropy",
        metrics=[CategoricalAccuracy(name="accuracy")]
    )

    return model

## 5.2 CNN (1D) Model Builder

In [ ]:
# --- Cell: CNN Model Builder ---
def CNNBuilder(optimizerStr, learningRate,
               conv_layers_structure, kernel_size,
               dense_layers_structure, dropout, applyDropout):
    """
    Builds a 1D Convolutional Neural Network (CNN) model dynamically.

    Args:
        optimizerStr (str): Name of the optimizer.
        learningRate (float): Learning rate.
        conv_layers_structure (list): List of integers defining filters for each Conv1D layer.
        kernel_size (int): Size of the 1D convolution window.
        dense_layers_structure (list): List of integers for the dense layers after convolution.
        dropout (float): Dropout rate for the dense layers.
        applyDropout (bool): Whether to apply dropout in the dense block.
    """
    optimizerCls = {
        "Adam": Adam, "RMSprop": RMSprop, "SGD": SGD,
        "Adagrad": Adagrad, "Adadelta": Adadelta, "Adamax": Adamax, "Nadam": Nadam,
    }

    layers = [Input(shape=input_shape)]

    # 1. Dynamic Convolutional Blocks
    # Each block consists of Conv1D -> MaxPooling1D
    for filters in conv_layers_structure:
        layers.append(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same'))
        layers.append(MaxPooling1D(pool_size=2))
        # Optional: You can add BatchNormalization() here if needed

    # 2. Transition to Dense Layers
    # GlobalAveragePooling1D is often better than Flatten for CNNs (reduces parameters significantly)
    layers.append(GlobalAveragePooling1D())

    # 3. Dynamic Dense Classification Head
    for units in dense_layers_structure:
        layers.append(Dense(units, activation='relu'))
        if applyDropout:
            layers.append(Dropout(dropout))

    # 4. Output Layer
    layers.append(Dense(NUM_CLASSES, activation="softmax"))

    # 5. Model Assembly
    model = Sequential(layers)

    model.compile(
        optimizer=optimizerCls[optimizerStr](learning_rate=learningRate),
        loss="categorical_crossentropy",
        metrics=[CategoricalAccuracy(name="accuracy")]
    )

    return model

## 5.3 Split data to train/val/test sets

In [ ]:
# --- 2. Dane ---
def CreateDataGenerators(batchSize, X, y):
    if y.ndim == 1:
        y_encoded = to_categorical(y, num_classes=NUM_CLASSES)
    else:
        y_encoded = y

    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y_encoded, test_size=0.3, random_state=42, stratify=y
    )
    y_stratify_temp = y[len(X_train):]

    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_stratify_temp
    )

    train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)) \
        .shuffle(buffer_size=len(X_train)).batch(batchSize).prefetch(tf.data.AUTOTUNE)

    val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)) \
        .batch(batchSize).prefetch(tf.data.AUTOTUNE)

    test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)) \
        .batch(batchSize)

    return train_ds, val_ds, test_ds

## 5.4 Optuna objective function

In [ ]:
#exp_name = "Optuna_Hybrid_Search_FFT_v1"

exp_name = "Optuna_Hybrid_Search_TIME_v2"

In [ ]:
def ObjectiveFunction(trial):
    # 1. Inicjalizacja W&B
    parts = exp_name.split('_')

    domain = parts[-2]  # "TIME"
    version = parts[-1] # "v2"

    run_name = f"trial_{trial.number}_{domain}{version}"

    run = wandb.init(
        project="UAV-FDI-Optimization",
        group=exp_name,
        name=run_name,
        job_type="hyperparam_opt",
        reinit=True
    )

    try:
        clear_session()

        # --- A. Parametry Wspólne ---
        # Te parametry są używane przez obie architektury

        # --- A. Zapisywanie Konfiguracji Danych (METADANE) ---
        # To zapisze się w bazie, ale Optuna nie będzie tego "losować"
        trial.set_user_attr("domain", DOMAIN)
        trial.set_user_attr("sensor_mode", SENSOR_MODE)
        trial.set_user_attr("input_shape", str(input_shape)) # Warto zamienić krotkę/listę na string dla bezpieczeństwa bazy
        trial.set_user_attr("output_shape", NUM_CLASSES)

        params = {
            "optimizer": trial.suggest_categorical("optimizer", ["Adam", "Nadam", "RMSprop", "SGD", "Adagrad"]),
            "learningRate": trial.suggest_float("learningRate", 1e-5, 1e-2, log=True),
            "batchSize": trial.suggest_categorical("batchSize", [64, 128, 256, 512]),
            "model_type": trial.suggest_categorical("model_type", ["MLP", "CNN"]), # Decyzja: która sieć?
            "epochs": 15, # Stała liczba epok dla porównania (możesz zwiększyć)
            "domain": DOMAIN,
            "sensor_mode": SENSOR_MODE,
            "input_shape": input_shape,
            "output_shape": NUM_CLASSES,
        }

        model = None

        # --- B. Rozgałęzienie (Conditional Logic) ---

        if params["model_type"] == "MLP":
            # === ŚCIEŻKA MLP ===
            dropout = trial.suggest_float("mlp_dropout", 0.1, 0.5, step=0.1)
            apply_dropout = trial.suggest_categorical("mlp_apply_dropout", [True, False])

            # Losowanie warstw Dense
            n_layers = trial.suggest_int("mlp_n_layers", 1, 4)
            hidden_structure = []
            for i in range(n_layers):
                units = trial.suggest_int(f"mlp_units_l{i}", 32, 512, step=32)
                hidden_structure.append(units)
                params[f"mlp_units_l{i}"] = units # Logujemy do W&B

            # Zapisujemy specyficzne parametry do słownika params
            params.update({
                "dropout": dropout,
                "applyDropout": apply_dropout,
                "structure": str(hidden_structure)
            })

            # Budujemy MLP
            model = MLPBuilder(
                optimizerStr=params["optimizer"],
                dropout=dropout,
                applyDropout=apply_dropout,
                learningRate=params["learningRate"],
                hidden_layers_structure=hidden_structure
            )

        else:
            # === ŚCIEŻKA CNN ===
            dropout = trial.suggest_float("cnn_dropout", 0.1, 0.5, step=0.1)
            apply_dropout = trial.suggest_categorical("cnn_apply_dropout", [True, False])
            kernel_size = trial.suggest_categorical("cnn_kernel_size", [5, 10, 100, 500])

            # 1. Losowanie warstw Conv
            n_conv = trial.suggest_int("cnn_n_conv_layers", 1, 4)
            conv_structure = []
            for i in range(n_conv):
                filters = trial.suggest_int(f"cnn_filters_l{i}", 32, 512, step=32)
                conv_structure.append(filters)
                params[f"cnn_filters_l{i}"] = filters

            # 2. Losowanie głowy Dense (Classification Head)
            n_dense = trial.suggest_int("cnn_n_dense_head", 1, 4)
            dense_structure = []
            for i in range(n_dense):
                units = trial.suggest_int(f"cnn_dense_l{i}", 64, 512, step=32)
                dense_structure.append(units)
                params[f"cnn_dense_l{i}"] = units

            # Zapisujemy parametry
            params.update({
                "dropout": dropout,
                "applyDropout": apply_dropout,
                "kernel_size": kernel_size,
                "conv_structure": str(conv_structure),
                "dense_structure": str(dense_structure)
            })

            # Budujemy CNN
            model = CNNBuilder(
                optimizerStr=params["optimizer"],
                learningRate=params["learningRate"],
                conv_layers_structure=conv_structure,
                kernel_size=kernel_size,
                dense_layers_structure=dense_structure,
                dropout=dropout,
                applyDropout=apply_dropout
            )

        # --- C. Logowanie Konfiguracji do W&B ---
        wandb.config.update(params)

        # --- D. Trening ---
        train_ds, val_ds, test_ds = CreateDataGenerators(params["batchSize"], X, y)

        history = model.fit(
            train_ds,
            epochs=params["epochs"],
            validation_data=val_ds,
            callbacks=[
                EarlyStopping(patience=8, restore_best_weights=True),
                WandbMetricsLogger(log_freq="epoch") # Loguje loss/acc do W&B
            ],
            verbose=0
        )

        # --- E. Ewaluacja ---
        loss, accuracy = model.evaluate(test_ds, verbose=0)

        # Logujemy finalny wynik testowy
        wandb.log({"test_accuracy": accuracy, "test_loss": loss})

        wandb.finish()
        return accuracy

    except Exception as e:
        print(f"!!! Błąd w próbie {trial.number}: {e}")
        wandb.finish(exit_code=1)
        return 0.0

## 5.5 Optuna study start

In [ ]:
# --- Start Badania Hybrydowego ---
db_folder = "History/PretrainedOptuna"
os.makedirs(db_folder, exist_ok=True)
storage_url = f"sqlite:///{db_folder}/PretrainedOptuna.db"

# Nowa nazwa dla badania hybrydowego
study_name = exp_name

print(f"Start badania: {study_name}")
study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_url,
    load_if_exists=True
)

# Wyłączamy logi W&B w konsoli (żeby było czytelniej)
os.environ["WANDB_SILENT"] = "true"

# Uruchamiamy np. 30 prób, żeby Optuna miała czas przetestować oba typy
study.optimize(ObjectiveFunction, n_trials=5, show_progress_bar=True)

# --- Cell: Download Database to Local PC ---
db_file = "PretrainedOptuna.db"
db_path = os.path.join(db_folder, db_file)

if os.path.exists(db_path):
    print(f"Pobieranie pliku: {db_path} ...")
    files.download(db_path)
else:
    print(f"Błąd: Plik {db_path} nie istnieje. Uruchom najpierw trening (Optunę).")

In [ ]:
print("Najlepsza próba:")
print(f"  Typ modelu: {study.best_params['model_type']}")
print(f"  Accuracy: {study.best_trial.value}")
print("  Parametry:", study.best_params)

# 6. SQL DATA

## 6.1 Download Database to Local PC

In [ ]:
# --- Cell: Download Database to Local PC ---
import os
from google.colab import files

# Ścieżka do Twojej bazy (zdefiniowana wcześniej w kodzie)
db_folder = "History/PretrainedOptuna"
db_file = "PretrainedOptuna.db"
db_path = os.path.join(db_folder, db_file)

if os.path.exists(db_path):
    print(f"Pobieranie pliku: {db_path} ...")
    files.download(db_path)
else:
    print(f"Błąd: Plik {db_path} nie istnieje. Uruchom najpierw trening (Optunę).")

## 6.3 Upload Database from Local PC

In [ ]:
# --- Cell: Upload Database from Local PC ---
import os
import shutil
from google.colab import files

# 1. Przygotuj strukturę folderów
db_folder = "History/PretrainedOptuna"
os.makedirs(db_folder, exist_ok=True)

print("Wgraj plik 'PretrainedOptuna.db' ze swojego komputera:")
uploaded = files.upload()

# 2. Przenieś wgrany plik do odpowiedniego folderu
for filename in uploaded.keys():
    if filename.endswith(".db"):
        source_path = filename
        destination_path = os.path.join(db_folder, filename)

        # Przenoszenie (nadpisze plik, jeśli już tam jest)
        shutil.move(source_path, destination_path)
        print(f"Sukces! Baza danych przywrócona do: {destination_path}")
        print("Możesz teraz uruchomić celę z ładowaniem Optuny (RUN_OPTIMIZATION = False).")
    else:
        print(f"Wgrano plik '{filename}', ale to nie wygląda na bazę danych (.db).")

## 6.3 SQLite data reading

### Database inspection

In [ ]:
# --- Cell: Inspect Database Content (Tables & Studies) ---
import sqlite3
import optuna
import pandas as pd
import os

# Konfiguracja ścieżki
db_folder = "History/PretrainedOptuna"
db_file = "PretrainedOptuna.db"
db_path = os.path.join(db_folder, db_file)
storage_url = f"sqlite:///{db_folder}/{db_file}"

if os.path.exists(db_path):
    print(f"📂 Analiza pliku bazy danych: {db_file}\n")

    # --- CZĘŚĆ 1: Lista Tabel SQL (Techniczna struktura) ---
    print("--- 1. Struktura Bazy Danych (Tabele SQL) ---")
    try:
        conn = sqlite3.connect(db_path)
        # Pobieramy nazwy tabel
        query = "SELECT name FROM sqlite_master WHERE type='table';"
        tables = pd.read_sql_query(query, conn)

        if not tables.empty:
            print(tables)
            print("\nWyjaśnienie najważniejszych tabel:")
            print(" - studies: Lista Twoich eksperymentów (np. v1, v2)")
            print(" - trials: Lista wszystkich prób (runów) ze wszystkich badań")
            print(" - trial_values: Wyniki (Accuracy) dla każdej próby")
            print(" - trial_params: Parametry (lr, dropout) dla każdej próby")
        else:
            print("⚠️ Brak tabel. Baza jest pusta.")
        conn.close()
    except Exception as e:
        print(f"❌ Błąd SQL: {e}")

    print("-" * 30)

# --- CZĘŚĆ 2: Lista Twoich Badań (Logiczna zawartość) ---
    print("\n--- 2. Zapisane Badania (Studies) ---")
    try:
        # Optuna ma funkcję do podsumowania wszystkich badań w pliku
        summaries = optuna.get_all_study_summaries(storage=storage_url)

        if summaries:
            # Tworzymy ładną tabelkę
            studies_data = []
            for i, s in enumerate(summaries):
                # POPRAWKA: Używamy enumerate zamiast s.study_id, które zostało usunięte w nowszej Optunie
                studies_data.append({
                    "Index": i,
                    "Nazwa Badania": s.study_name,
                    "Liczba Prób": s.n_trials,
                    "Start": s.datetime_start.strftime("%Y-%m-%d %H:%M") if s.datetime_start else "N/A"
                })

            df_studies = pd.DataFrame(studies_data)
            display(df_studies)
        else:
            print("⚠️ W tej bazie nie ma jeszcze żadnych badań.")

    except Exception as e:
        print(f"❌ Błąd Optuny: {e}")

else:
    print(f"❌ Plik {db_path} nie istnieje.")

### Displaying experiments trials

In [ ]:
# --- KONFIGURACJA ŚCIEŻEK ---
# Upewnij się, że ścieżka i nazwa badania są identyczne jak w etapie treningu
db_folder = "History/PretrainedOptuna"
db_file = "PretrainedOptuna.db"
storage_url = f"sqlite:///{db_folder}/{db_file}"
study_name = exp_name

# --- SPRAWDZENIE CZY BAZA ISTNIEJE ---
if not os.path.exists(os.path.join(db_folder, db_file)):
    print(f"BŁĄD: Nie znaleziono pliku bazy danych w: {db_folder}/{db_file}")
    print("Upewnij się, że uruchomiłeś wcześniej trening.")
else:
    print(f"Ładowanie badania '{study_name}' z bazy danych...")

    # Ładujemy istniejące badanie (nie tworzymy nowego)
    try:
        study = optuna.load_study(
            study_name=study_name,
            storage=storage_url
        )

        # --- 1. Wyświetlenie Najlepszego Wyniku ---
        if len(study.trials) > 0:
            best_trial = study.best_trial
            print(f"\nZnaleziono {len(study.trials)} zakończonych prób.")
            print(f"NAJLEPSZY WYNIK (Test Accuracy): {best_trial.value:.4f}")
            print("   Parametry zwycięzcy:")
            for key, value in best_trial.params.items():
                print(f"     - {key}: {value}")

            # --- 2. Tabela TOP 5 Modeli ---
            print("\nTabela 5 Najlepszych Modeli:")
            df_results = study.trials_dataframe()

            # Sortujemy malejąco po wyniku (Accuracy)
            df_top5 = df_results.sort_values(by='value', ascending=False).head(5)

            # Lista kolumn, które chcemy wyświetlić (jeśli istnieją w bazie)
            # Optuna dodaje prefiks 'params_' do nazw parametrów
            wanted_cols = [
                'number', 'value', 'params_model_type', 'duration'
            ]

            # Wybieramy tylko te kolumny, które faktycznie są w DataFrame
            # (np. params_mlp_n_layers może nie istnieć, jeśli wylosowano same CNN)
            cols_to_show = [c for c in wanted_cols if c in df_top5.columns]

            try:
                display(df_top5[cols_to_show])
            except NameError:
                print(df_top5[cols_to_show].to_string())

        else:
            print("Badanie istnieje, ale nie zawiera żadnych zakończonych prób.")

    except KeyError:
        print(f"Nie znaleziono badania o nazwie '{study_name}' w pliku .db.")

In [ ]:
df_top5.columns

In [ ]:
study.study_name

# 7. Retrain Selected Model - W&B integration

In [ ]:
# === CONFIGURATION ===
# Set to None to automatically select the best trial from the study.
# Set to an integer (e.g., 5) to retrain a specific trial number.
SELECTED_TRIAL_NUMBER = None

# How many epochs to train for the final run (usually more than in search)
RETRAIN_EPOCHS = 10
# =====================

# 1. Retrieve parameters
if SELECTED_TRIAL_NUMBER is None:
    target_trial = study.best_trial
    print(f"Selected BEST trial (ID: {target_trial.number}) with val_acc: {target_trial.value:.4f}")
else:
    # Find trial by number
    target_trial = next(t for t in study.trials if t.number == SELECTED_TRIAL_NUMBER)
    print(f"Selected specific trial (ID: {target_trial.number}) with val_acc: {target_trial.value:.4f}")


optuna_params = target_trial.params
user_attributes = target_trial.user_attrs
print("Parameters:", optuna_params)

# Łączymy parametry z Optuny z Twoimi stałymi parametrami
combined_config = {
    "retrain_epochs": RETRAIN_EPOCHS,
    "trial_id": target_trial.number,
    "optuna_original_test_acc": target_trial.value,
    # Nazwa badania, z którego pochodzi model
    "source_study_name": study.study_name,
    # Rozpakowanie parametrów z Optuny:
    **optuna_params,
    **user_attributes
}

# 1. Inicjalizacja W&B dla finalnego treningu
run = wandb.init(
    project="UAV-FDI-Optimization",   # Ten sam projekt co wcześniej
    group="Final_Training",           # Nowa nazwa grupy (żeby oddzielić od searcha)
    ## ----
    name="Time_MLP_v1",               # Unikalna nazwa tego konkretnego przebiegu
    ## ----
    config=combined_config,
    reinit=True
)

#print("W&B zinicjalizowane. Config:", combined_config)
print("W&B zinicjalizowane.")

# --- 3. Reconstruct Architecture & Build Model ---
model_type = optuna_params["model_type"]
final_model = None # Zmieniam nazwę na final_model dla porządku

if model_type == "MLP":
    hidden_structure = []
    n_layers = optuna_params["mlp_n_layers"]
    for i in range(n_layers):
        hidden_structure.append(optuna_params[f"mlp_units_l{i}"])

    print(f"Building MLP with structure: {hidden_structure}")

    final_model = MLPBuilder(
        optimizerStr=optuna_params["optimizer"],
        dropout=optuna_params["mlp_dropout"],
        applyDropout=optuna_params["mlp_apply_dropout"],
        learningRate=optuna_params["learningRate"],
        hidden_layers_structure=hidden_structure
    )

elif model_type == "CNN":
    conv_structure = []
    n_conv = optuna_params["cnn_n_conv_layers"]
    for i in range(n_conv):
        conv_structure.append(optuna_params[f"cnn_filters_l{i}"])

    dense_structure = []
    n_dense = optuna_params["cnn_n_dense_head"]
    for i in range(n_dense):
        dense_structure.append(optuna_params[f"cnn_dense_l{i}"])

    print(f"Building CNN with Conv: {conv_structure} and Dense: {dense_structure}")

    final_model = CNNBuilder(
        optimizerStr=optuna_params["optimizer"],
        learningRate=optuna_params["learningRate"],
        conv_layers_structure=conv_structure,
        kernel_size=optuna_params["cnn_kernel_size"],
        dense_layers_structure=dense_structure,
        dropout=optuna_params["cnn_dropout"],
        applyDropout=optuna_params["cnn_apply_dropout"]
    )

# 4. Prepare Data
print(f"Preparing data with Batch Size: {optuna_params['batchSize']}")
train_ds, val_ds, test_ds = CreateDataGenerators(optuna_params['batchSize'], X, y)

# 5. Callbacks for Retraining
callbacks_list = [
    EarlyStopping(patience=15, restore_best_weights=True, monitor="val_accuracy"),
    ReduceLROnPlateau(factor=0.2, patience=10, min_lr=1e-6, monitor="val_loss"),
    WandbMetricsLogger(log_freq="epoch")
]


# 6. Start Training
print(f"Rozpoczynam trening finalny ({RETRAIN_EPOCHS} epok)...")

history = final_model.fit(
    train_ds,
    epochs=RETRAIN_EPOCHS,
    validation_data=val_ds,
    callbacks=callbacks_list,
    verbose=1
)


# 7. Zapisz Model Checkpoint (Raz, po zakończeniu)
save_folder = "final_models_ckpt"
os.makedirs(save_folder, exist_ok=True)
model_filename = f"{optuna_params["model_type"]}_{user_attributes["domain"]}_cl{user_attributes["output_shape"]}_trial_{target_trial.number}_from_{study.study_name}.keras"
checkpoint_path = os.path.join(save_folder, model_filename)

final_model.save(checkpoint_path)
print(f"Model zapisany lokalnie jako: {model_filename}")

artifact_name = f"model_{optuna_params['model_type']}_{user_attributes['domain']}"

model_artifact = wandb.Artifact(
    name=artifact_name,
    type="model",
    #description=f"Model",
    metadata=combined_config  # <--- Bardzo przydatne! Konfig jest przyklejony do modelu
)


model_artifact.add_file(checkpoint_path)


run.log_artifact(model_artifact)

print(f"Model wysłany jako Artifact: {artifact_name}")

# Wrzuć plik modelu do chmury W&B
# wandb.save(model_filename)
# print("Model wysłany do Weights & Biases.")

# 4. Wygeneruj i wyślij Schemat architektury
plot_filename = "model_architecture.png"
try:
    # Tworzymy plik graficzny ze schematem
    plot_model(
        final_model,
        to_file=plot_filename,
        show_shapes=True,
        show_layer_names=True,
        expand_nested=True
    )

    # Logujemy obrazek do dashboardu
    wandb.log({"model_chart": wandb.Image(plot_filename)})
    print("Schemat architektury (plot_model) wysłany do W&B.")

except Exception as e:
    print(f"Nie udało się wygenerować plot_model (może brakować graphviz): {e}")

# 5. Ewaluacja końcowa i zamknięcie
loss, accuracy = final_model.evaluate(test_ds, verbose=0)
wandb.log({"test_accuracy": accuracy, "test_loss": loss})


print(f"Wynik końcowy na teście: {accuracy:.4f}")

wandb.finish()

# Retrain Selected Model - without online logger

In [ ]:
# === CONFIGURATION ===
# Set to None to automatically select the best trial from the study.
# Set to an integer (e.g., 5) to retrain a specific trial number.
SELECTED_TRIAL_NUMBER = 10

# How many epochs to train for the final run (usually more than in search)
RETRAIN_EPOCHS = 100
# =====================

# 1. Retrieve parameters
if SELECTED_TRIAL_NUMBER is None:
    target_trial = study.best_trial
    print(f"Selected BEST trial (ID: {target_trial.number}) with val_acc: {target_trial.value:.4f}")
else:
    # Find trial by number
    target_trial = next(t for t in study.trials if t.number == SELECTED_TRIAL_NUMBER)
    print(f"Selected specific trial (ID: {target_trial.number}) with val_acc: {target_trial.value:.4f}")

params = target_trial.params
user_attributes = target_trial.user_attrs
print("Parameters:", params)

# 2. Reconstruct Architecture & Build Model
model_type = params["model_type"]
model = None

if model_type == "MLP":
    # --- Reconstruct MLP Structure ---
    hidden_structure = []
    n_layers = params["mlp_n_layers"]
    for i in range(n_layers):
        hidden_structure.append(params[f"mlp_units_l{i}"])

    print(f"Building MLP with structure: {hidden_structure}")

    model = MLPBuilder(
        optimizerStr=params["optimizer"],
        dropout=params["mlp_dropout"],
        applyDropout=params["mlp_apply_dropout"],
        learningRate=params["learningRate"],
        hidden_layers_structure=hidden_structure
    )

elif model_type == "CNN":
    # --- Reconstruct CNN Structure ---
    conv_structure = []
    n_conv = params["cnn_n_conv_layers"]
    for i in range(n_conv):
        conv_structure.append(params[f"cnn_filters_l{i}"])

    dense_structure = []
    n_dense = params["cnn_n_dense_head"]
    for i in range(n_dense):
        dense_structure.append(params[f"cnn_dense_l{i}"])

    print(f"Building CNN with Conv: {conv_structure} and Dense: {dense_structure}")

    model = CNNBuilder(
        optimizerStr=params["optimizer"],
        learningRate=params["learningRate"],
        conv_layers_structure=conv_structure,
        kernel_size=params["cnn_kernel_size"],
        dense_layers_structure=dense_structure,
        dropout=params["cnn_dropout"],
        applyDropout=params["cnn_apply_Dropout"]
    )

# 3. Prepare Data (using the trial's batch size)
print(f"Preparing data with Batch Size: {params['batchSize']}")
train_ds, val_ds, test_ds = CreateDataGenerators(params['batchSize'], X, y)

# 4. Callbacks for Retraining

save_folder = "final_models_ckpt"
os.makedirs(save_folder, exist_ok=True)
model_filename = f"{optuna_params["model_type"]}_{user_attributes["domain"]}_cl{user_attributes["output_shape"]}_trial_{target_trial.number}_from_{study.study_name}.keras"
checkpoint_path = os.path.join(save_folder, model_filename)

callbacks_list = [
    ModelCheckpoint(checkpoint_path, save_best_only=True, monitor="val_accuracy", mode="max", verbose=1),
    EarlyStopping(patience=8, restore_best_weights=True, monitor="val_accuracy"),
    ReduceLROnPlateau(factor=0.2, patience=3, min_lr=1e-6, monitor="val_loss")
]

# 5. Start Training
print(f"\nStarting retraining for {RETRAIN_EPOCHS} epochs...")
history = model.fit(
    train_ds,
    epochs=RETRAIN_EPOCHS,
    validation_data=val_ds,
    callbacks=callbacks_list,
    verbose=1
)

# 6. Final Evaluation
print("\n--- Final Evaluation on Test Set ---")
loss, accuracy = model.evaluate(test_ds)
print(f"Final Test Accuracy: {accuracy:.4f}")
print(f"Model saved to: {checkpoint_path}")

# 7. Plotting Results
plt.figure(figsize=(12, 5))

# Accuracy Plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title(f'Accuracy (Trial {target_trial.number})')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Loss Plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title(f'Loss (Trial {target_trial.number})')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Model getting from w&b

In [ ]:
import wandb
import tensorflow as tf
import glob

run = wandb.init(project="UAV-FDI-Optimization", job_type="inference")

# Pobierasz ZAWSZE najnowszą wersję tego typu modelu:
artifact = run.use_artifact('USER_NAME/UAV-FDI-Optimization/model_MLP_time:latest')


# artifact.download() zwraca ścieżkę do folderu
model_dir = artifact.download()

# Szukamy dowolnego pliku kończącego się na .keras w tym folderze
files = glob.glob(os.path.join(model_dir, "*.keras"))

if files:
    model_path = files[0] # Bierzemy pierwszy znaleziony plik
    print(f"Znaleziono model: {model_path}")
    model = tf.keras.models.load_model(model_path)
else:
    raise FileNotFoundError(f"Nie znaleziono pliku .keras w folderze {model_dir}")